# Figure 8

## Imports

In [ ]:
import dolfin  
import matplotlib.pyplot as plt
import numpy
import sympy
import sys

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

from matplotlib.collections import PatchCollection

## Defining Geometry and material parameters

In [ ]:
seeds_filename = "Fig8-seeds.dat"
mesh_filebasename = "Fig8-mesh"

domain_y = 1.
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vol = (xmax - xmin)*(ymax - ymin)
dV = dolfin.Measure("dx",domain=mesh)
Phi_s0 = dolfin.assemble(1*dV)/vol

vertices = numpy.array([[xmin, ymin],
                     [xmax, ymin],
                     [xmax, ymax],
                     [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

In [ ]:
Es = 1.
nus = 0.2
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

## Defining kinematics

In [ ]:
alpha_x, alpha_y, alpha_xy, alpha_yx = sympy.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x, epsilon_y, epsilon_xy = sympy.symbols('epsilon_x epsilon_y epsilon_xy')
epsilon = sympy.symbols('epsilon')
alpha_x = epsilon + 1
alpha_y = epsilon + 1

C00, C01, C10, C11 = sympy.symbols('C00 C01 C10 C11')
C_CM = sympy.Matrix([[C00, C01],
                     [C10, C11]])
I_C_CM = sympy.trace(C_CM) 
II_C_CM = (sympy.trace(C_CM)**2 - sympy.trace(C_CM**2))/2 
III_C_CM = sympy.det(C_CM) 
J_CM = sympy.sqrt(III_C_CM)
p = sympy.symbols('p')

F = sympy.Matrix(
        [[alpha_x,    0  ],\
        [   0   , alpha_y]])
J = F.det()
C = F.T * F
E = (C - sympy.eye(2))/2

## Computing macroscopic model response

In [ ]:
def macroscopic_model(mat_params):
    homogenization_problem = dmech.HomogenizationProblem(
        dim=2,
        mesh=mesh,
        mat_params=mat_params["parameters"],
        vertices=vertices,
        vol=vol,
        bbox=bbox)

    lmbda_tilde, mu_tilde = homogenization_problem.get_lambda_and_mu()
    beta_tilde = mu_tilde/2
    alpha_tilde = lmbda_tilde/4


    W_skel = beta_tilde * (I_C_CM - 2 - 2 * sympy.ln(J_CM)) + alpha_tilde * (J_CM**2 - 1 - 2 * sympy.ln(J_CM)) 
    Sigma_CM = 2*sympy.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

    Sigma = Sigma_CM.subs(list(zip(C_CM, C))).doit().as_explicit() 
    sigma = F * Sigma * F.T / J 

    return sigma

mat_params = {"model":"CGNH", "parameters":{"E":0.5, "nu":nus}}
sigma_macro_1 = macroscopic_model(mat_params).subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":1, "nu":nus}}
sigma_macro_2 = macroscopic_model(mat_params).subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":2, "nu":nus}}
sigma_macro_3 = macroscopic_model(mat_params).subs(p, 0)


epsilon_vals = numpy.linspace(0, 0.5, 100)

lam = sympy.lambdify(epsilon, sigma_macro_1[0, 0], modules=['numpy'])
sigma_vals_1 = lam(epsilon_vals)

lam = sympy.lambdify(epsilon, sigma_macro_2[0, 0], modules=['numpy'])
sigma_vals_2 = lam(epsilon_vals)

lam = sympy.lambdify(epsilon, sigma_macro_3[0, 0], modules=['numpy'])
sigma_vals_3 = lam(epsilon_vals)

## Computing microscopic model response

In [ ]:
qois_filename = "Fig8-qois.dat"
res_basename = "Fig8"

load_params = {}
load_params["pf"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0
load_params["U_bar_00"] = 0.5
load_params["U_bar_11"] = 0.5

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":0.5, "nu":nus}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_1 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_Es_1 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":1, "nu":nus}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_2 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_Es_2 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":2, "nu":nus}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_3 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_Es_3 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

In [ ]:
class MulticolorPatch(object):
    def __init__(self, colors):
        self.colors = colors
        
# Defining a handler for the MulticolorPatch object
class MulticolorPatchHandler(object):
    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        width, height = handlebox.width, handlebox.height
        patches = []
        for i, c in enumerate(orig_handle.colors):
            patches.append(plt.Rectangle([width/len(orig_handle.colors) * i - handlebox.xdescent, 
                                          -handlebox.ydescent],
                           width / len(orig_handle.colors),
                           height, 
                           facecolor=c, 
                           edgecolor='none'))

        patch = PatchCollection(patches,match_original=True)

        handlebox.add_artist(patch)
        return patch

# ------ choose some colors
colors1 = ['#084594', '#99000D']
colors2 = ['#4292C6', '#EF3B2C']
colors3 = ['#9ECAE1', '#FC9272']

# ------ create a dummy-plot (just to show that it works)
f, ax = plt.subplots()
ax.plot(eps_glob_1, sigma_glob_Es_1, c=colors1[0])
ax.plot(eps_glob_2, sigma_glob_Es_2, c=colors2[0])
ax.plot(eps_glob_3, sigma_glob_Es_3, c=colors3[0])
ax.plot(epsilon_vals, sigma_vals_1, c=colors1[1])
ax.plot(epsilon_vals, sigma_vals_2, c=colors2[1])
ax.plot(epsilon_vals, sigma_vals_3, c=colors3[1])

# ------ get the legend-entries that are already attached to the axis
h, l = ax.get_legend_handles_labels()

# ------ append the multicolor legend patches
h.append(MulticolorPatch(colors1))
l.append(r'$E_s = 0.5~kPa$')

h.append(MulticolorPatch(colors2))
l.append(r'$E_s = 1~kPa$')

h.append(MulticolorPatch(colors3))
l.append(r'$E_s = 2~kPa$')

# ------ create the legend
f.legend(h, l, loc='upper left', 
         handler_map={MulticolorPatch: MulticolorPatchHandler()}, 
         bbox_to_anchor=(.125,.875))

plt.xlim(0, 0.5)
plt.ylim(0, 0.175)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
# plt.savefig('stress_strain_micro_macro_Es.pdf',bbox_inches='tight')
plt.show()

### Stress-strain comparision micro-Macro for differnent $\nu_s$

In [ ]:
dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":Es, "nu":0.1}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_1 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_nu_01 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":Es, "nu":0.2}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_2 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_nu_02 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

dmech.run_HollowBox_MicroPoroHyperelasticity(
    dim=2,
    mesh=mesh,
    mat_params={"model":"CGNH", "parameters":{"E":Es, "nu":0.4}},
    load_params=load_params,
    step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.05, "dt_max":0.05},
    res_basename=res_basename,
    verbose=1)
qois_vals = numpy.loadtxt(qois_filename)
qois_name_list = open(qois_filename).readline().split()
eps_glob_3 = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]
sigma_glob_nu_03 = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]

mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.1}}
sigma_macro_1 = macroscopic_model(mat_params).subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.2}}
sigma_macro_2 = macroscopic_model(mat_params).subs(p, 0)
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":0.4}}
sigma_macro_3 = macroscopic_model(mat_params).subs(p, 0)

epsilon_vals = numpy.linspace(0, 0.5, 100)

lam = sympy.lambdify(epsilon, sigma_macro_1[0, 0], modules=['numpy'])
sigma_macro_1 = lam(epsilon_vals)

lam = sympy.lambdify(epsilon, sigma_macro_2[0, 0], modules=['numpy'])
sigma_macro_2 = lam(epsilon_vals)

lam = sympy.lambdify(epsilon, sigma_macro_3[0, 0], modules=['numpy'])
sigma_macro_3 = lam(epsilon_vals)

In [ ]:
f, ax = plt.subplots()
ax.plot(eps_glob_1, sigma_glob_nu_01, c=colors1[0])
ax.plot(eps_glob_2, sigma_glob_nu_02, c=colors2[0])
ax.plot(eps_glob_3, sigma_glob_nu_03, c=colors3[0])
ax.plot(epsilon_vals, sigma_macro_1, c=colors1[1])
ax.plot(epsilon_vals, sigma_macro_2, c=colors2[1])
ax.plot(epsilon_vals, sigma_macro_3, c=colors3[1])

# ------ get the legend-entries that are already attached to the axis
h, l = ax.get_legend_handles_labels()

# ------ append the multicolor legend patches
h.append(MulticolorPatch(colors1))
l.append(r'$\nu_s = 0.1$')

h.append(MulticolorPatch(colors2))
l.append(r'$\nu_s = 0.2$')

h.append(MulticolorPatch(colors3))
l.append(r'$\nu_s = 0.4$')

# ------ create the legend
f.legend(h, l, loc='upper left', 
         handler_map={MulticolorPatch: MulticolorPatchHandler()}, 
         bbox_to_anchor=(.125,.875))


plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x,~E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma~(kPa)$', fontsize=16)
plt.xlim(0, 0.5)
plt.ylim(0, 0.175)
# plt.savefig('stress_strain_micro_macro_nus.pdf',bbox_inches='tight')
plt.show()